In [23]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

import re

import json

In [24]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

sleep(1)
input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
input_case_reason.send_keys("殺人")
input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")

sleep(2)

# 送出搜尋
search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
search_button.click()
sleep(3)

In [25]:
#切到iframe中，抓page網址
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
driver.switch_to.frame(iframe)
sleep(2)
#main-section
page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
print(page_url)

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in


#測試一下標籤
driver.get(page_url)
cssSelector = 'a#hlTitle'
case_url = driver.find_element(By.CSS_SELECTOR, cssSelector)
case_url.click()
# 如果這邊click之後，沒有等JSt生成，就直接page_source抓動態html的話，會抓不到，用標籤判斷根本沒用，標籤會在，只是沒有內容，看有沒有其他隱性等待的方法(這樣比較保險)，不然就要強制ˋ等待了，或使隱性等待?(但他是如何判斷加載完成的...)
sleep(1)
try:
    # 看有沒有抓到js生成的側攔
    sidebarSelector = 'div.rela-area.col-xs-4'
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
    (By.CSS_SELECTOR, sidebarSelector)
    ))

    #有的話，就取得動態html
    case_html = driver.page_source
    soup = bs(case_html, "lxml")

except TimeoutException:
    print("not found...element")


# side_bar = soup.select_one('div.rela-area.col-xs-4')
# print(side_bar.get_text())
linesSelector = 'ul.rela-law'
lines = soup.select_one(linesSelector)
print(lines.get_text())



side_bar = soup.select_one('div.rela-area.col-xs-4')
print(side_bar.get_text())

In [26]:
page_url_list = [] 
regex_page_url = "&page=[\d]*"
test = re.split(regex_page_url, page_url)
for each_page in range(1,26):
    page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 

In [27]:
# 放所有case的資料
all_case_list = []

In [28]:
# 加try excecpt
# 要設一些如果沒抓到東西時的條件，比如歷史審判可能會沒有
# 然後改移下一開始的wait反正就是等旁偏那個javascript生出來的測攔出來就好
# 開始爬case，這是每一頁的網址
for page in page_url_list:
    driver.get(page)
    try:
        cssSelector = 'a#hlTitle'
        WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, cssSelector)
        ))

        case_url = driver.find_elements(By.CSS_SELECTOR, cssSelector)
    except TimeoutException:
        print("cases not found")
        break

    # cases找到後開始一個一個點
    # 這是每一頁當中的至多20筆的資料    
    for case in case_url:
        case.click()
        #要邊要等網頁跑一下，不然javascript一樣出不來(暫時想不出更好的辦法，就先用強制等待)
        sleep(3)

        #取得動態html
        case_html = driver.page_source
        soup = bs(case_html, "lxml")

        allSelector = 'div.col-td'
        all = soup.select(allSelector)


        linesSelector = 'ul.rela-law'
        lines = soup.select_one(linesSelector)

        #目前出錯的都是這一塊
        history_judgement = soup.select("div#JudHis div.panel-body ul li")
        history_judgement_list = []
        try:
            for index in range(0, len(history_judgement)):
                history_judgement_list.append({
                    "case_id":str(history_judgement[index].get_text()),
                    "url":"https://judgment.judicial.gov.tw/FJUD/"+history_judgement[index].select_one('a')['href']
                })
        except TypeError: 
            # 如果沒有就pass就好，history_judgement就給她空list
            pass
        all_case_list.append({
        "case_id":str(all[0].get_text().strip()),
        "date":str(all[1].get_text().strip()),
        "simple_reason":str(all[2].get_text().strip()),
        "context":str(all[3].get_text()),
        "laws":lines,
        "history_judgement":history_judgement_list
        })

        driver.back()


In [29]:
print(all_case_list[44]['laws'])

<ul class="rela-law"><li>刑事訴訟法 第 159.5、299 條（106.11.16）</li><li>中華民國刑法 第 1、3、19、25、38、87、271 條（105.11.30）</li></ul>


In [30]:
print(len(all_case_list))

500



with open ('./cases_500', 'w', encoding='utf-8') as f:
    f.writelines(all_case_list)
    f.close()

In [37]:
import pandas as pd

In [38]:
# 就寫成dataframe再轉成csv八
tt = pd.DataFrame(all_case_list)
tt.to_csv('./cases_500.csv')

In [44]:
# 這樣也可以轉json?? 亂碼...懶得搞了
tt = pd.DataFrame(all_case_list)
tt.to_json('./cases_500.json')